# Karar Ağacı

Karar Ağaçları ile Titanic'de hayatta kalanların modellenmesi amaçlanmamıştır.


# Giriş

Mevcut verilerden gerçekten içgörüler elde etmemiz gerektiğinde **Karar Ağaçları** gibi şeffaf, anlaşılması kolay modeller kullanmamız gerekir.

Doğrudan bazı görevler için kullanılacak bir model oluşturmamız veya **yalnızca son sonuçlarını göstermemiz gerekiyorsa** yeterli başarıma sahip bir "kara kutu (blackbox)" oluşturan [*Derin Öğrenme*] gibi gelişmiş teknikler kullanarak, karmaşık görevlerde yüksek başarımlar elde edebiliriz. 

Ancak her zaman göz önünde bulundurulması gereken şey :

### **karmaşıklık / doğruluk dengesi**

karmaşık teknikler yalnızca önemli iyileştirmeler sağladıkları takdirde kullanılmalıdır. **Çünkü daha basit modeller overfittinge daha az eğilimlidir ve daha iyi genelleme yeteneğine sahiptir.**

**Karar Ağaçlarının ana dezavantajları**

*   Overfitting eğilimleri
*   Açgözlü öğrenme (greedy) algoritmalarını kullanılması
*   Öznitelikler arasındaki karmaşık ilişkileri kavrayamamaları


**Karar Ağaçları'na dair bu kısa girişten sonra, bunları Titanik veri kümesine nasıl uygulayacağımızı göreceğiz.** 

1.   veri setini hazırlayacağız ve en alakalı özellikleri tartışacağız.
2.   aşırı uydurmayı önlemek için en iyi ağaç derinliğini bulacağız.
3.   nihai modeli oluşturacağız.
4.   ortaya çıkan ağacın nasıl görselleştirileceğini göreceğiz.


## Titanic veri kümesini hazırlama ##


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#ROOT_DIR = "/content/drive/MyDrive/CASGEM-Egitim/Egitim-Part1/Day7-DecisionTree/notebooks"
ROOT_DIR = "https://media.githubusercontent.com/media/yapay-ogrenme/casgem-eu-project-training-on-data-mining/main/PART1/Day7-DecisionTree/notebooks"

DATASET_PATH = ROOT_DIR + "/datasets/titanic/"

In [ ]:
# Imports needed for the script
import numpy as np
import pandas as pd
import re
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont

# Loading the data
df_train = pd.read_csv(DATASET_PATH + 'train.csv')
df_test = pd.read_csv(DATASET_PATH + 'test.csv')

# Kolay erişim için test yolcu kimliklerimizi saklayın
PassengerId = df_test['PassengerId']

# Eğitim kümesine genel bakış
df_train.head(3)

Bu genel bakış sayesinde, veri setimizin bazı işlemlere ihtiyacı olduğunu görebiliriz. 

Bu konuda zaten genişletilmiş çalışmalar var, bu yüzden sadece en iyi yaklaşımlardan birini kullanıyoruz.

In [ ]:
# Daha sonra ilginç özellikleri araştırırken ihtiyaç duymamız ihtimaline karşı orijinal veri kümesini kopyalayın
# UYARI: Sadece referans vermek yerine veri çerçevesini gerçekten kopyalamaya dikkat edin
# "original_train = train", train değişkenine bir referans oluşturacak ('train'deki değişiklikler' original_train 'için geçerli olacaktır)

original_train = df_train.copy()# 'Copy ()' kullanılması, aynı değerlerle farklı bir nesne oluşturarak veri kümesini klonlamaya izin verir

full_data = [df_train, df_test]

In [ ]:
# Titanik'te bir yolcunun kabini olup olmadığını söyleyen özellik
df_train['Has_Cabin'] = df_train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
df_test['Has_Cabin'] = df_test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)


In [ ]:
# SibSp ve Parch'ın bir kombinasyonu olarak FamilySize adlı yeni özellik oluşturun
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

In [ ]:
# Yeni özellik oluşturma FamilySize'dan yararlanılacak IsAlone
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

In [ ]:
# Embarked sütunundaki tüm BOŞLUKLARI kaldırın
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

In [ ]:
# Fare sütunundaki tüm BOŞLUKLARI kaldırın
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(df_train['Fare'].median())

In [ ]:
# Age sütunundaki tüm BOŞLUKLARI kaldırın
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    # Next line has been improved to avoid warning
    dataset.loc[np.isnan(dataset['Age']), 'Age'] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)

In [ ]:
# Yolcu isimlerinden başlıkları çıkarmak için işlevi tanımlayın
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
   # Başlık varsa, çıkartın ve iade edin.
    if title_search:
        return title_search.group(1)
    return ""


for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)

In [ ]:
# Tüm yaygın olmayan başlıkları tek bir gruplama "Rare" olarak gruplandırın
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

In [ ]:
for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping titles
    title_mapping = {"Mr": 1, "Master": 2, "Mrs": 3, "Miss": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] 						        = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] 							        = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] ;

In [ ]:
# Özellik seçimi: artık ilgili bilgileri içermeyen değişkenleri kaldırın
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
df_train = df_train.drop(drop_elements, axis = 1)
df_test  = df_test.drop(drop_elements, axis = 1)

## İşlenmiş verileri görselleştirme ##

In [ ]:
df_train.head(3)

Veri kümemiz, yalnızca sayısal değerler ve potansiyel olarak anlamlı özelliklerle artık eskisinden çok daha temiz. 

Şimdi veri setimizdeki tüm öznitelikler arasındaki korelasyon tablosunu çizerek değişkenlerimiz arasındaki ilişkiyi inceleyelim :


In [ ]:
colormap = plt.cm.viridis
plt.figure(figsize=(12,12))
plt.title('Correlation of Features', y=1.05, size=15)
sns.heatmap(df_train.astype(float).corr().abs(),linewidths=0.1,vmax=1.0, square=True, cmap=colormap, linecolor='white', annot=True)

Bu ısı haritası, ilk gözlem olarak çok kullanışlıdır çünkü her özelliğin tahmini değeri hakkında kolayca fikir edinebilirsiniz. 

Bu durumda, **Sex(Cinsiyet)** ve **Title(Ünvan)** öznitelikleri, hedef değişken **Survived (Hayatta Kalan)**  ile en yüksek korelasyonları (mutlak olarak) gösterir (Sırasıyla 0,54 ve 0,49)

Ancak her ikisi (**Sex(Cinsiyet)** ve **Title(Ünvan)**) arasındaki mutlak korelasyon da çok yüksektir (0.86, veri setimizdeki en yüksek değer), bu nedenle muhtemelen aynı bilgiyi taşıyorlar. Bu yüzden ikisini aynı model için girdi olarak kullanmak iyi bir fikir değil. 


## **Title vs. Sex**
-------

Özellikleri ve sınıfla olan ilişkilerini gruplayarak ve her grup için bazı temel istatistikleri hesaplayarak kolayca karşılaştırabilirsiniz.

"Survived" bir ikili sınıf (0 veya 1) olduğundan, Title özelliğine göre gruplandırılan bu ölçümler şunları temsil eder:

*   **MEAN: hayatta kalma oranı**
*   **COUNT: toplam gözlem**
*   **SUM: hayatta kalan insanlar**   

title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5} 

In [ ]:
df_train[['Title', 'Survived']].groupby(['Title'], as_index=False).agg(['mean', 'count', 'sum'])



Survived bir ikili özellik olduğundan, Sex özelliğine göre gruplandırılan bu ölçümler şunları temsil eder:

*   **MEAN: hayatta kalma oranı**
*   **COUNT: toplam gözlem**
*   **SUM: hayatta kalan insanlar**    

sex_mapping = {{'female': 0, 'male': 1}} 

In [ ]:
df_train[['Sex', 'Survived']].groupby(['Sex'], as_index=False).agg(['mean', 'count', 'sum'])

# Survived bir ikili özellik olduğundan, Sex özelliğine göre gruplandırılan bu ölçümler şunları temsil eder:
     # MEAN: hayatta kalma oranı
     # COUNT: toplam gözlem
     # SUM: hayatta kalan insanlar 
    


**Title** özelliğinin **Sex** den daha yararlı görünüyor. 

Bunun nedeni, **Title'ın** çoğu durumda dolaylı olarak **Sex** hakkında bilgi içermesidir. 

Bunu doğrulamak için, **Title**'a göre gruplanmış **Sex** dağılımını kontrol edebiliriz.

Sex ikili bir özellik olduğundan, Title özelliğine göre gruplandırılan bu ölçümler şunları temsil eder:    

*   **MEAN: erkeklerin yüzdesi**
*   **COUNT: toplam gözlem**
*   **SUM: erkeklerin sayısı**

    
    


In [ ]:
# Her başlık için cinsiyet dağılımını kontrol etmek için 'orijinal_train' veri çerçevemizi kullanalım.
# Original_train veri kümesindeki değişiklikleri önlemek için tekrar copy () kullanıyoruz

title_and_sex = original_train.copy()[['Name', 'Sex']]

# Create 'Title' feature
title_and_sex['Title'] = title_and_sex['Name'].apply(get_title)

# Map 'Sex' as binary feature
title_and_sex['Sex'] = title_and_sex['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

# Table with 'Sex' distribution grouped by 'Title'
title_and_sex[['Title', 'Sex']].groupby(['Title'], as_index=False).agg(['mean', 'count', 'sum'])



Tek bir gözlem ('Dr'ünvanlı bir kadın) haricinde, belirli bir Title için tüm gözlemlerin aynı Sex'i paylaştığını görüyoruz. Bu nedenle Title özelliği, Sex'te bulunan tüm bilgileri yakalar. 

**Ek olarak, Title, bireylerin yaş, sosyal sınıf, kişilik, ... gibi diğer özelliklerini yakalayarak görevimiz için daha değerli olabilir.**

Sex ve Title özelliklerinin bu derinlemesine analizi sayesinde, Sex özelliğinin Survived sınıfıyla korelasyonu daha yüksek olsa bile, Unvanın Sex bilgilerini taşıdığı için daha zengin bir özellik olduğunu ancak başka özellikler de eklediğini gördük. 

**Bu nedenle, Title büyük olasılıkla son karar ağacımızdaki ilk özellik olacak ve bu ilk bölünmeden sonra Sex'i işe yaramaz hale getirecek.**


## Gini Impurity ##

**Karar Ağaçları algoritmalarının amacı her zaman ağacın her bir düğümü için en iyi bölünmeyi bulmaktır.** 

Ancak belirli bir bölünmenin "iyiliğini" ölçmek öznel bir sorudur, bu nedenle pratikte bölünmeleri değerlendirmek için farklı ölçütler kullanılır. 

Yaygın olarak kullanılan bir ölçü **[Information Gain]** 'dir. Diğer bir yaygın ölçü olan **[Gini Impurity]** 'dir.

Gini Impurity, bir dizi öğenin bozukluğunu ölçer. Bir elemanın kümedeki tüm sınıfların dağılımına göre rastgele etiketlendiği varsayılarak bir elemanın yanlış etiketlenmesi olasılığı olarak hesaplanır. 

**Karar Ağaçları**, ortaya çıkan iki düğümde Gini Impurity'i en çok azaltan ayrımı bulmaya çalışacaktır.




In [ ]:
# Define function to calculate Gini Impurity
def get_gini_impurity(survived_count, total_count):
    survival_prob = survived_count/total_count

    not_survival_prob = (1 - survival_prob)

    random_observation_survived_prob = survival_prob

    random_observation_not_survived_prob = (1 - random_observation_survived_prob)

    mislabelling_survided_prob = not_survival_prob * random_observation_survived_prob
    mislabelling_not_survided_prob = survival_prob * random_observation_not_survived_prob

    gini_impurity = mislabelling_survided_prob + mislabelling_not_survided_prob
    
    return gini_impurity

Örnek olarak **Sex** ve **Title** özelliklerimizi kullanalım ve her bir bölünmenin genel ağırlıklı Gini Impurity'i ne kadar azaltacağını hesaplayalım. 

İlk olarak, eğitim veri setimizdeki 891 gözlemin tamamını içeren başlangıç düğümünün Gini Impurity hesaplamamız gerekir. 

Yalnızca 342 gözlem hayatta kaldığından, hayatta kalma olasılığı yaklaşık %38'dir.

In [ ]:
# Gini Impurity of starting node
gini_impurity_starting_node = get_gini_impurity(342, 891)
gini_impurity_starting_node

Şimdi her iki bölünmeyi de simüle edeceğiz, ortaya çıkan düğümlerin safsızlığını hesaplayacağız ve ardından her bir bölünmenin aslında safsızlığı ne kadar azalttığını ölçmek için bölünmeden sonra ağırlıklı Gini Safsızlığını elde edeceğiz.

**Cinsiyete** göre ayrılırsak, aşağıdaki iki düğüme sahip oluruz:

  - Erkeklerle düğüm: Sadece 109'u hayatta kalan 577 gözlem
  - Kadınlarla düğüm: 233 hayatta kalan 314 gözlem

In [ ]:
# 'Erkek' gözlemleri için düğümde Gini Impurity azalması
gini_impurity_men = get_gini_impurity(109, 577)
gini_impurity_men

In [ ]:
# 'Kadın' gözlemleri için düğümde Gini Impurity azalması
gini_impurity_women = get_gini_impurity(233, 314)
gini_impurity_women

In [ ]:
# Düğüm cinsiyete göre bölünürse Gini Impurity azalması
men_weight = 577/891
women_weight = 314/891
weighted_gini_impurity_sex_split = (gini_impurity_men * men_weight) + (gini_impurity_women * women_weight)

sex_gini_decrease = weighted_gini_impurity_sex_split - gini_impurity_starting_node
sex_gini_decrease

**Title == 1 (== Mr)** ile bölersek, aşağıdaki iki düğüme sahip oluruz:

  - Sadece Mr. olan düğüm: Sadece 81'i hayatta kalan 517 gözlem
  - Diğer başlıklara sahip düğüm: 261 hayatta kalan 374 gözlem

In [ ]:
# Title == 1 == Mr ile gözlemler için düğümde Impurity azalması
gini_impurity_title_1 = get_gini_impurity(81, 517)
gini_impurity_title_1

In [ ]:
# Title =! 1 =! Mr ile gözlemler için düğümde Impurity azalması
gini_impurity_title_others = get_gini_impurity(261, 374)
gini_impurity_title_others

In [ ]:
# Düğüm Title == 1 == Mr ile gözlemler için bölünürse, Gini Impurity azalması
title_1_weight = 517/891
title_others_weight = 374/891
weighted_gini_impurity_title_split = (gini_impurity_title_1 * title_1_weight) + (gini_impurity_title_others * title_others_weight)

title_gini_decrease = weighted_gini_impurity_title_split - gini_impurity_starting_node
title_gini_decrease

Title özelliğinin Gini Impurity azaltmada Cinsiyetten biraz daha iyi olduğunu görüyoruz. 

**Bu, önceki analizimizi doğruluyor ve artık Title'ın ilk bölünmede kullanılacağından eminiz.**

Bu nedenle, bilgiler Title özelliğine zaten dahil edildiği için cinsiyet ihmal edilecektir.

## Çapraz Doğrulama yardımıyla en iyi ağaç derinliğini bulma ##

Karar ağaçları söz konusu olduğunda, **'max_depth'** parametresi, modelin her bir tahmin için kullanacağı maksimum öznitelik sayısını belirler (veri kümesindeki mevcut özelliklerin sayısına kadar). Bu parametre için en iyi değeri bulmanın iyi bir yolu, mümkün olan tüm derinlikleri yinelemek ve doğruluğu **Çapraz Doğrulama (Cross Validation)** gibi sağlam bir yöntemle ölçmektir.





In [ ]:
cv = KFold(n_splits=10)            # Desired number of Cross Validation folds
accuracies = list()
max_attributes = len(list(df_test))
depth_range = range(1, max_attributes + 1)

# 1'den maksimum özniteliğe maks_depthleri test etme
# Uncomment prints for details about each Cross Validation pass
for depth in depth_range:
    fold_accuracy = []
    tree_model = tree.DecisionTreeClassifier(max_depth = depth)
    # print("Current max depth: ", depth, "\n")
    for train_fold, valid_fold in cv.split(df_train):
        f_train = df_train.loc[train_fold] # Extract train data with cv indices
        f_valid = df_train.loc[valid_fold] # Extract valid data with cv indices

        model = tree_model.fit(X = f_train.drop(['Survived'], axis=1), 
                               y = f_train["Survived"]) # We fit the model with the fold train data
                               
        valid_acc = model.score(X = f_valid.drop(['Survived'], axis=1), 
                                y = f_valid["Survived"])# We calculate accuracy with the fold validation data
        fold_accuracy.append(valid_acc)

    avg = sum(fold_accuracy)/len(fold_accuracy)
    accuracies.append(avg)
    # print("Accuracy per fold: ", fold_accuracy, "\n")
    # print("Average accuracy: ", avg)
    # print("\n")
    
# Just to show results conveniently
df = pd.DataFrame({"Max Depth": depth_range, "Average Accuracy": accuracies})
df = df[["Max Depth", "Average Accuracy"]]
print(df.to_string(index=False))

Bu nedenle, **en iyi maksimum derinlik parametresi 3** gibi görünmektedir ve modeli daha fazla veri ile beslemek, muhtemelen overfit nedeniyle en kötü sonuçlara yol açmaktadır. 

Bu nedenle, son modelimiz için max_depth parametresi olarak 3 kullanacağız.

## Final Tree ##

In [ ]:
# Create Numpy arrays of train, test and target (Survived) dataframes to feed into our models
# Modellerimizi beslemek için Numpy dizileri oluşturalım: eğitim, test ve target (Survived) dataframes
y_train = df_train['Survived']
x_train = df_train.drop(['Survived'], axis=1).values 
x_test = df_test.values

# Create Decision Tree with max_depth = 3
decision_tree = tree.DecisionTreeClassifier(max_depth = 3)
decision_tree.fit(x_train, y_train)

# Predicting results for test dataset
#y_pred = decision_tree.predict(x_test)


# Export our trained model as a .dot file
with open("tree1.dot", 'w') as f:
     f = tree.export_graphviz(decision_tree,
                              out_file=f,
                              max_depth = 3,
                              impurity = True,
                              feature_names = list(df_train.drop(['Survived'], axis=1)),
                              class_names = ['Died', 'Survived'],
                              rounded = True,
                              filled= True )
        
#Convert .dot to .png to allow display in web notebook
check_call(['dot','-Tpng','tree1.dot','-o','tree1.png'])

# Annotating chart with PIL
img = Image.open("tree1.png")
draw = ImageDraw.Draw(img)
font = ImageFont.truetype('/usr/share/fonts/truetype/liberation/LiberationSerif-Bold.ttf', 26)
draw.text((10, 0), # Drawing offset (position)
          '"Title <= 1.5" corresponds to "Mr." title', # Text to draw
          (0,0,255), # RGB desired color
          font=font) # ImageFont object with desired font
img.save('sample-out.png')
PImage("sample-out.png")

# Code to check available fonts and respective paths
# import matplotlib.font_manager
# matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')

In [ ]:
acc_decision_tree = round(decision_tree.score(x_train, y_train) * 100, 2)
acc_decision_tree

**Nihayet Karar Ağacımız çizildi! Eğitim veri setinde % 82,38 doğruluk elde etti. Grafiğin nasıl okunacağını açıklamaya başlayalım.**

Her düğümün ilk satırı (son satırdakiler hariç) bölme koşulunu 
**"feature <= value"** biçiminde gösterir.

Daha sonra, düğümün Gini Impurtiy değerini buluyoruz. "Samples" ise düğümde bulunan gözlemlerin sayısıdır.

**"Value", örneklerin sınıf dağılımını gösterir ([count non_survived, count survived]).**

Son olarak, "class", her düğümün baskın sınıfına karşılık gelir ve modelimiz bir gözlemi bu şekilde sınıflandırır. 

Modelimiz bu nedenle 4 basit kuralla özetlenebilir:

* Gözlemimiz de "Mr" Title'ını içeriyorsa, onu hayatta kalmamış olarak sınıflandırırız.

* "Mr." Title'ını içermiyorsa ve FamilySize 4 veya daha küçükse, o zaman hayatta kalmış olarak sınıflandırıyoruz.

* "Mr." Title'ını içermiyorsa, FamilySize 4'ten fazla ve Pclass 2 veya daha küçükse, o zaman hayatta kalan olarak sınıflandırıyoruz.

* "Mr" Title'ını içermiyorsa, FamilySize 4'ten büyükse ve Pclass 2'den fazlaysa, onu hayatta kalmamış olarak sınıflandırıyoruz.


Bu kurallar sayesinde gemi enkazı hakkında bazı içgörüler elde edebiliriz. 

Görünüşe göre "Baylar (Mr.)" unvanlarını onurlandırdılar ve "Usta" veya "Dr." gibi daha egzotik unvanlarla kadın ve erkekler lehine kendilerini feda ettiler.

Ayrıca, daha küçük ailelerin hayatta kalma şanslarının daha yüksek olduğunu da not edebiliriz, çünkü daha büyük aileler bir arada kalmaya veya kayıp üyeleri aramaya çalıştıkları için cankurtaran sandallarında yer kalmamış olabilir. 

Son olarak, 3. sınıf yolcuların hayatta kalma şanslarının da daha az olduğunu gözlemleyebiliriz, bu nedenle muhtemelen üst sosyal sosyal sınıflara ait yolcular ayrıcalıklıydı veya sadece 3. sınıf kabinler cankurtaran botlarından daha uzaktaydı.